Documentation for the data can be found [here](http://ww2.amstat.org/publications/jse/v19n3/Decock/DataDocumentation.txt). 

I take a lot of inspiration from:
* [The Kaggle Learn: Machine Learning lessons by Dan Becker](https://www.kaggle.com/learn/machine-learning)
*  [Stacked Regressions by Serigne](https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard)
* [Comprehensive data exploration with Python by  Pedro Marcelino](https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python)

Please pay attention to the first block of code. At the top I define the number of theads to be used as a global variable. I have a Ryzen 7 2700X, which is an 8-core CPU with a base clock speed of 3.7 Ghz. If you try running this notebook, you may run into issues with the number of threads you can run simultaneous or in the run time you experience. This notebook in it's current state still takes about 30 minutes for me.

In [ ]:
nthreads = 8

import numpy as np 
import pandas as pd
import sklearn

import os
print(os.listdir("../input"))


In [ ]:
import seaborn as sns
import matplotlib as plt
from plotnine import *
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from scipy.stats import norm    # used in plotting

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
import lightgbm as lgb

In [ ]:
trainData = pd.read_csv("../input/train.csv")
testData = pd.read_csv("../input/test.csv")
testID = testData.Id
trainData.head()

# Data Imputation

The documentation refers to 5 outliers in this dataset, of which 3 are true outliers. A plot of sale price against ground living area is encouraged. Below, 4 clear outliers, all above 4000 sq ft ground living area are clear. Two of which seem due to large houses while another two are anomalous. We eliminate the anomalous values. The last point may be lost in the test data set or culled for being an outlier, though this cannot be determined without the `SalePrice` values redacted for the test set.

In [ ]:
sns.scatterplot(data=trainData,x="GrLivArea",y="SalePrice")

In [ ]:
outliers = ((trainData.GrLivArea > 4000) & (trainData.SalePrice < 5E5))

In [ ]:
trainData = trainData[~(outliers)]
sns.scatterplot(data=trainData,x="GrLivArea",y="SalePrice")

Before continuing we should look at which variables are highly correlated with our SalePrice target. A correlation matrix in the form of a heatmap provides this by reading off color values from the last row of the chart while additionally providing information about correlations between variables.

In [ ]:
f, ax = plt.pyplot.subplots(figsize=(12, 9))
sns.heatmap(trainData.corr(),square=True,cmap="YlGnBu")

From this we see the most important variables seem to be `OverallQual`, `TotalBsmtSF`, `1stFlrSF`, `GrLivArea`, `GarageCars`, and `GarageArea`. We should pay extra attention to these later before training the model.
Of these, we see that the pairs (`TotalBsmtSF`, `1stFlrSF`) and (`GarageCars`,`GarageArea`) are highly correlated. This is expected, as the basement and first floor are matched to the size of the foundation, and the number of cars that can fit in a garage is going to be dependent on how big the garage is.

In [ ]:
impCols = ["OverallQual","TotalBsmtSF", "1stFlrSF", "GrLivArea", "GarageCars", "GarageArea"]

Before checking for missing data, see what the documentation has to say about each column and convert null values to their actual indicator. This will be packed into a function in order to pipeline when making predictions on test data. Here we list the columns transformed:
* Alley: no alley
*  Bsmt\*: no basement
* FireplaceQu: no fireplace
* Garage\*: no garage
    + expect missing values for GarageYrBlt
* PoolQC: no pool
* Fence: no fence
* MiscFeature: no such item as an elevator, tennis court, second garage, etc...

In [ ]:
def nullfill(in_df):
    df = in_df.copy()
    df.Alley = df.Alley.fillna("None")
    df.BsmtQual = df.BsmtQual.fillna("None")
    df.BsmtCond = df.BsmtCond.fillna("None")
    df.BsmtExposure = df.BsmtExposure.fillna("None")
    df.BsmtFinType1 = df.BsmtFinType1.fillna("None")
    df.BsmtFinType2 = df.BsmtFinType2.fillna("None")
    df.FireplaceQu = df.FireplaceQu.fillna("None")
    df.GarageType = df.GarageType.fillna("None")
    df.GarageFinish = df.GarageFinish.fillna("None")
    df.GarageQual = df.GarageQual.fillna("None")
    df.GarageCond = df.GarageCond.fillna("None")
    df.PoolQC = df.PoolQC.fillna("None")
    df.Fence = df.Fence.fillna("None")
    df.MiscFeature = df.MiscFeature.fillna("None")
    return(df)

In [ ]:
cleanData = nullfill(trainData)
cleanNa = cleanData.isnull().sum().sort_values(ascending=False)

cleanNa.head(5)

As expected, the `GarageYrBlt`has null values (as many as were filled from the other garage-related variables).    
`MasVnr` is short for Masonry Veneer, which should already have a value "None". Still the equality between the missing type and area seems to indicate that this should be fixed anyways.

Imputring data for the `Electrical` feature is less intuitive. We could expect the type of electrical system to vary with build date, though updates are possible. Let's see if that's true.

In [ ]:
( ggplot(trainData, aes("YearBuilt"))
   + geom_histogram(bins = 50) 
   + facet_wrap("Electrical")
)

From the above plot it seems safe to impute null values in `Electrical` as "SBrkr" seeing as that type is dominant over the time period where null values exist (towards the end of year built data) as well as over the entire range of observed values for year built.

(Maybe fix this graph to be proportion rather than raw count)

In [ ]:
def nullfill2(in_df):
    df = nullfill(in_df)
    df.MasVnrType = df.MasVnrType.fillna("None")
    df.MasVnrArea = df.MasVnrArea.fillna(0)
    df.Electrical = df.Electrical.fillna("SBrkr")
    return(df)

In [ ]:
cleanData = nullfill2(trainData)
cleanNa = cleanData.isnull().sum().sort_values(ascending=False)

cleanNa.head(5)

A quick check of the test data shows that this cleaning is insufficient, and all cases should be covered. This requires going back and stepping through each class to provide a standard for null inputs.

In [ ]:
cleanTest = nullfill2(testData)
testCleanNA = cleanTest.isnull().sum().sort_values(ascending=False)

testCleanNA.head(5)

Performing the same analysis we did on the `Electrical` feature, we see that houses in this dataset predominantly have central air. This leads us to impute the `CentralAir` value as "y".

In [ ]:
( ggplot(trainData, aes("YearBuilt"))
   + geom_histogram(bins = 50) 
   + facet_wrap("CentralAir")
)

We are still left with uncertainty as to what to do with the `GarageYrBlt` and `LotFrontage` features. 

Concerning the `GarageYrBlt` variable: The plot below shows that the garage is largely built with the house and of those that were not, there is no immediate correlation. Imputing a value of 0 may be dangerous for a continuous variable in a linear regression type model. Additionally, our correlation matrix above shows that this variable may explain some variance in the data so it may not be wise to delete this variable. Consequently, it seems like a good choice to impute values of 0 if we commit to only use decision tree-type models. 

`LotFrontage` values could mean that the lot doesn't actually have any direct street access. In depth analysis could involve looking at these specific houses to determine what is going on, or seeing correlation with alley access. From there we can impute a value of zero or the modal value for each neighborhood. During the model training phase these two methods will be compared.

In [ ]:
sns.scatterplot(data=trainData,x="YearBuilt",y="GarageYrBlt")

In [ ]:
(ggplot(trainData,aes("LotFrontage"))
  + geom_histogram(bins=30)
  + facet_wrap("Neighborhood")
)

In [ ]:
frontageNull = trainData.LotFrontage.isnull().groupby(trainData["Neighborhood"]).sum()
frontageNull.sort_values(ascending=False).plot.bar()

In [ ]:
totalByNeighborhood = trainData.groupby("Neighborhood").count().Id

totalByNeighborhood.sort_values(ascending=False).plot.bar()

In [ ]:
fig1 = sns.scatterplot(x=totalByNeighborhood,y=frontageNull)
fig1.set(xlabel="Total Observations",ylabel="Number missing")
fig1

For the MSZoning and Utilities features we impute on the modal value for each neighborhood, creating a dictionary. We create an additional dictionary to compare our different `LotFrontage` imputations as well.

In [ ]:
zoning = trainData.groupby("Neighborhood").MSZoning.apply(lambda x: x.value_counts().sort_values().index[0]).to_dict()
utilities = trainData.groupby("Neighborhood").Utilities.apply(lambda x: x.value_counts().sort_values().index[0]).to_dict()
frontage = trainData.groupby("Neighborhood").LotFrontage.apply(lambda x: x.value_counts().sort_values().index[0]).to_dict()

In [ ]:
# values that null is filled with "None"
fillNone = ["Alley","BsmtQual","BsmtCond","MasVnrType","BsmtExposure","BsmtFinType1","BsmtFinType2","FireplaceQu","GarageType","GarageFinish","GarageQual",
           "GarageCond","PoolQC","Fence","MiscFeature","MasVnrType"]

# For categorical data, it is 
fillZero = ["BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF","BsmtFullBath","BsmtHalfBath","FullBath","HalfBath","BedroomAbvGr","2ndFlrSF","GrLivArea",
           "MasVnrArea","GarageArea","GarageCars", "GarageYrBlt"]

def imputeVals0(in_df):
    df = in_df.copy()
    for i in fillNone:
        df[i] = df[i].fillna("None")
    for i in fillZero:
        df["null_%s" % (i)] = df[i].isnull()                           # mark which zeros are imputed
        df[i] = df[i].fillna(0)
    df.Electrical = df.Electrical.fillna("SBrkr")
    df.Functional = df.Functional.fillna("Typ")                        # Documentation instructs to assume "typical" unless otherwise noted
    df.CentralAir = df.CentralAir.fillna("Y")
    df["null_LotFrontage"] = df.LotFrontage.isnull()
    df.LotFrontage = df.LotFrontage.fillna(0)                             
    df.MSZoning = df.MSZoning.fillna(df.Neighborhood.map(zoning))
    df.Utilities = df.Utilities.fillna(df.Neighborhood.map(utilities))
    df.KitchenQual = df.KitchenQual.fillna("Po")                      #one house missing kitchen data
    df.SaleType = df.SaleType.fillna("Oth")                           # only one missing value, fill the already defined "other"
    df.Exterior1st = df.Exterior1st.fillna("Other")
    df.Exterior2nd = df.Exterior2nd.fillna("Other")                  # the same house is responsible for the missing exterior 1st and 2nd, other is predefined
    df = df.drop(columns=["Id"])
    return(df)

def imputeVals1(in_df):
    df = in_df.copy()
    for i in fillNone:
        df[i] = df[i].fillna("None")
    for i in fillZero:
        df["null_%s" % (i)] = df[i].isnull()
        df[i] = df[i].fillna(0)
    df.Electrical = df.Electrical.fillna("SBrkr")
    df.Functional = df.Functional.fillna("Typ")                        
    df.CentralAir = df.CentralAir.fillna("Y")
    df.LotFrontage = df.LotFrontage.fillna(df.Neighborhood.map(frontage))            # This is the only line different in these two functions, maybe a more elegant solution is possible               
    df.MSZoning = df.MSZoning.fillna(df.Neighborhood.map(zoning))
    df.Utilities = df.Utilities.fillna(df.Neighborhood.map(utilities))
    df.KitchenQual = df.KitchenQual.fillna("Po")                     
    df.SaleType = df.SaleType.fillna("Oth")                           
    df.Exterior1st = df.Exterior1st.fillna("Other")
    df.Exterior2nd = df.Exterior2nd.fillna("Other")                 
    df = df.drop(columns=["Id"])
    return(df)

In [ ]:
impData0 = imputeVals0(trainData)
impData1 = imputeVals1(trainData)

print("There are %d nulls left in the training data" % impData1.isnull().sum().sum())             # Check if both axes can be summed in one function call
print("There are %d nulls left in the testing data" % imputeVals0(testData).isnull().sum().sum())

# Feature Engineering

Here we go through the necessary steps of checking the assumptions of most models as well as transformations to optimize model performance. It is worth noting that on a decision tree-type model, renormalization will have little effect.

The plot below shows that the `SalePrice` target variable deviates greatly from a normal distribution. We can attribute this partially to the lower bound of 0 and no upper bound to the value of a home, causing a positive skew.

It is worth noting that the kurtosis produced by the default kurtosis function is the excess kurtosis where the normal distribution produces a value zero.

In [ ]:
sns.distplot(trainData["SalePrice"], fit=norm)
print("Skew: %.3f" % (trainData.SalePrice.skew()))
print("Kurtosis: %.3f" % (trainData.SalePrice.kurtosis()))

The scipy function `normaltest` uses these metrics to produce a statistic that is the sum of the squared skew and kurtosis as well as a p-value for a 2-sided $\chi^2$ test of normality. Below, the p-value indicates an unfathomably small probability that this sample was drawn from a normal distribution. To improve this, a bit of engineering is in order. A $log$ transformation tempers positive skewness while normalization often improves model performance.

In [ ]:
from scipy.stats import normaltest

normaltest(trainData.SalePrice.values)

In [ ]:
normedPrice = np.log(trainData.SalePrice)
sp_mean = np.mean(normedPrice)
sp_std = np.std(normedPrice)
normedPrice = (normedPrice - sp_mean) / sp_std

sns.distplot(normedPrice, fit=norm)

In [ ]:
normaltest(normedPrice)

This visually looks much better and the $\chi^2$ test shows greatly improved results but it still fails test of normality. More engineering may be necessary to improve performance in the future.

Recall we earmarked some features as being important predictors of our target feature:

In [ ]:
impCols

Good model performance can depend on the distribution of these values so we renormalize these as well before training. The normaltest performed for the target variable shows that; of our variables of interest; `TotalBsmtSF`, `1stFlrSF`, and `GrLivArea` are all in need of renormalization. Before performing renormalization it's important to note that our imputation procedure for continuous values included imputing zero, particularly for `TotalBsmtSF` where we recognized that houses don't necessarily have basements.  Blindly repeating our previous treatment of `SalePrice` will yield an error because $log(0)$ is undefined. 

In [ ]:
normedCols = impData0[impCols]

for col in impCols:
    print("%s pval: %.3e" % (col, normaltest(normedCols[col].values)[1]))

In [ ]:
temp = np.log(normedCols["1stFlrSF"])
temp_mean, temp_std = (np.mean(temp),np.std(temp))
temp = (temp - temp_mean)/temp_std
normedCols["1stFlrSF"] = temp

flr1_mean, flr1_std = (temp_mean, temp_std)          # save these values for the transform in the pipeline

normaltest(temp)

In [ ]:
temp = np.log(normedCols["GrLivArea"])
temp_mean, temp_std = (np.mean(temp),np.std(temp))
temp = (temp - temp_mean)/temp_std
normedCols["GrLivArea"] = temp

gflr_mean, gflr_std = (temp_mean, temp_std)    

normaltest(temp)

In [ ]:
normedCols.loc[normedCols.TotalBsmtSF > 0, "TotalBsmtSF"] = np.log(normedCols.TotalBsmtSF)      # this must be done with loc

In [ ]:
sns.distplot(normedCols.loc[normedCols.TotalBsmtSF > 0, "TotalBsmtSF"], fit=norm)

In [ ]:
print("Skew: %.3f" % normedCols.loc[normedCols.TotalBsmtSF > 0, "TotalBsmtSF"].skew())
print("Kurtosis: %.3f" % normedCols.loc[normedCols.TotalBsmtSF > 0, "TotalBsmtSF"].kurtosis())
normaltest(normedCols.loc[normedCols.TotalBsmtSF > 0, "TotalBsmtSF"].values)

Despite the poor p-value the distribution of nonzero values is visually improved. We forgo renormalization to avoid overlapping these values with imputed zeros.

Now wrap this up in a function for pipelining.

In [ ]:
def scalePrice(in_df):
    df = in_df.copy()
    df.SalePrice = np.log(trainData.SalePrice)
    df.SalePrice = (df.SalePrice - sp_mean)/sp_std
    return(df)

# This function will be necessary to extract predicted values
def invPrice(df):
    return( np.exp((df*sp_std)+sp_mean))

def featureFix(in_df):
    df = in_df.copy()
    temp = np.log(df["1stFlrSF"])
    temp = (temp - flr1_mean)/flr1_std
    df["1stFlrSF"] = temp
    temp = np.log(df["GrLivArea"])
    temp = (temp - gflr_mean)/gflr_std
    df["GrLivArea"] = temp
    df.loc[df.TotalBsmtSF > 0, "TotalBsmtSF"] = np.log(df.TotalBsmtSF)
    return(df)

For numerical data that is categorical, make sure that the type is String instead of Int

In [ ]:
trainData.columns

In [ ]:
catCol = ["MSSubClass","OverallCond","YrSold","MoSold","GarageYrBlt","YearBuilt","YearRemodAdd"]

def makeStr(in_df):
    df = in_df.copy()
    for col in catCol:
        df[col] = df[col].astype(str)
    return(pd.get_dummies(df))                          # convert to one-hot encoding

In [ ]:
def split_xy(in_df):
    y = in_df.SalePrice
    x = in_df.drop(columns=["SalePrice"])
    return(x,y)

# Model Building

First define a few functions for reuseability. 

We will use K-fold cross validation both in basic model testing and parameter searching, see documentation:
 - [KFold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)
 - [cross_val_score](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)
 - [GridSearchCV](http://scikit-learn.org/dev/modules/generated/sklearn.model_selection.GridSearchCV.html)

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error

train_x0, train_y = split_xy(makeStr(featureFix(scalePrice(impData0))))
train_x1, _ = split_xy(makeStr(featureFix(scalePrice(impData1))))

kf = KFold(n_splits=10, shuffle = True).get_n_splits(train_x0)

def rmse(model,x,y = train_y):
    rmse= np.sqrt(-cross_val_score(model, x.values, y.values, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

In [ ]:
# Note: I'm far more familiar with parallelizing in C++ and Scala, this code is likely not a canonical implementation

from pathos.multiprocessing import ProcessingPool as Pool

def helper_itModel(modelConstructor, data0, data1):
    model0 = modelConstructor
    model1 = modelConstructor
    cv0 = rmse(model0,data0)
    cv1 = rmse(model1,data1)
    return (np.mean(cv0),np.mean(cv1),np.std(cv0),np.std(cv1))

#### This is where the number of threads is used  ####
def itModel(n, modelConstructor, data0, data1, threads = nthreads):
    def tempFunc(modelConstructor):
        return helper_itModel(modelConstructor,data0,data1)
    procList = []
    for i in range(n):
        procList.append(modelConstructor)
    with Pool(processes=threads) as pool:
        multiOut = pool.map(tempFunc,procList)
    means = np.zeros(shape=(2,n))
    stds = np.zeros(shape=(2,n))
    for i in range(n):
        means[0,i] = multiOut[i][0]
        means[1,i] = multiOut[i][1]
        stds[0,i] = multiOut[i][2]
        stds[1,i] = multiOut[i][3]
    return(means,stds)

In [ ]:
from scipy.stats import ttest_ind

def ttest_model(n, modelConstructor, data0, data1, threads = nthreads):
    means, _ = itModel(n, modelConstructor, data0, data1, nthreads)
    ttest = ttest_ind(means[0,:],means[1,:])
    diff = np.mean(means[0,:]) - np.mean(means[1,:])
    return (ttest,diff)

# Model and Imputation Comparison

As promised, here is the comparison between imputing zero on LotFrontage (index zero) and the more complicated method of imputing on the modal value in each neighborhood (index 1.) We'll also look into the efficacy of flagging where values were imputed in the data set for decision trees, random forests, and gradient boosted trees using XG Boost.

tree1_ttest, tree1_diff = ttest_model(50,DecisionTreeRegressor(), train_x0, train_x1)

print(tree1_diff)
print(tree1_ttest)

rf_ttest, rf_diff   = ttest_model(50,RandomForestRegressor(), train_x0, train_x1)

print(rf_diff)
print(rf_ttest)

Interestingly, imputing zero on the LotFrontage feature works slightly better (statistically significant) for a decision tree regressor but has an insignificant effect  for a random forest regressor. 

In [ ]:
def imputeVals_noflag_0(in_df):
    df = in_df.copy()
    for i in fillNone:
        df[i] = df[i].fillna("None")
    for i in fillZero:
        #df["null_%s" % (i)] = df[i].isnull()                           # mark which zeros are imputed
        df[i] = df[i].fillna(0)
    df.Electrical = df.Electrical.fillna("SBrkr")
    df.Functional = df.Functional.fillna("Typ")                        # Documentation instructs to assume "typical" unless otherwise noted
    df.CentralAir = df.CentralAir.fillna("Y")
    #df["null_LotFrontage"] = df.LotFrontage.isnull()
    df.LotFrontage = df.LotFrontage.fillna(0)                             
    df.MSZoning = df.MSZoning.fillna(df.Neighborhood.map(zoning))
    df.Utilities = df.Utilities.fillna(df.Neighborhood.map(utilities))
    df.KitchenQual = df.KitchenQual.fillna("Po")                      #one house missing kitchen data
    df.SaleType = df.SaleType.fillna("Oth")                           # only one missing value, fill the already defined "other"
    df.Exterior1st = df.Exterior1st.fillna("Other")
    df.Exterior2nd = df.Exterior2nd.fillna("Other")                  # the same house is responsible for the missing exterior 1st and 2nd, other is predefined
    df = df.drop(columns=["Id"])
    return(df)

def imputeVals_noflag_1(in_df):
    df = in_df.copy()
    for i in fillNone:
        df[i] = df[i].fillna("None")
    for i in fillZero:
        #df["null_%s" % (i)] = df[i].isnull()
        df[i] = df[i].fillna(0)
    df.Electrical = df.Electrical.fillna("SBrkr")
    df.Functional = df.Functional.fillna("Typ")                        
    df.CentralAir = df.CentralAir.fillna("Y")
    df.LotFrontage = df.LotFrontage.fillna(df.Neighborhood.map(frontage))            # This is the only line different in these two functions, maybe a more elegant solution is possible               
    df.MSZoning = df.MSZoning.fillna(df.Neighborhood.map(zoning))
    df.Utilities = df.Utilities.fillna(df.Neighborhood.map(utilities))
    df.KitchenQual = df.KitchenQual.fillna("Po")                     
    df.SaleType = df.SaleType.fillna("Oth")                           
    df.Exterior1st = df.Exterior1st.fillna("Other")
    df.Exterior2nd = df.Exterior2nd.fillna("Other")                 
    df = df.drop(columns=["Id"])
    return(df)

In [ ]:
train_x_noflag_0, _ = split_xy(makeStr(featureFix(scalePrice(imputeVals_noflag_0(trainData)))))
train_x_noflag_1, _ = split_xy(makeStr(featureFix(scalePrice(imputeVals_noflag_1(trainData)))))

ttest_model(n = 500, modelConstructor = DecisionTreeRegressor(), data0 = train_x0, data1 = train_x_noflag_0)

This confirms the suspicion that adding those flags does little to improve the quality of the base decision tree, despite being statistically significant. Maybe we would use them if we were playing for inches.  
How about with a random forest?

rf_0_flags_test, rf_0_flags_diff = ttest_model(n = 50, modelConstructor = RandomForestRegressor(), data0 = train_x0, data1 = train_x_noflag_0)
print(rf_0_flags_diff)
print(rf_0_flags_test)

Flags appear to do nothing for the random forest.

It is necessary to randomly generate the state for XGBRegressor because it is otherwise defaulted to a value of zero. Otherwise we will produce a set of identical values for each set.

In [ ]:
from random import randint
def nextRand(): 
    return randint(0,10000)

In [ ]:
# LONG RUN TIME
#xgb_noflag_test, xgb_noflag_diff = ttest_model(n = 50, modelConstructor = XGBRegressor(random_state=nextRand()), data0 = train_x_noflag_0, data1 = train_x_noflag_1, threads = nthreads)
#print(xgb_noflag_diff)
#print(xgb_noflag_test)

$\Delta$ = 0.0014741226742583935  
Ttest_indResult(statistic=185888033141677.97, pvalue=0.0)  

This result is likely due to the the random state seeding of XGBoost however a workaround is still a work in progress. In other languages call by name evaluation of nextRand() would take care of the problem. It's also possible that the pickling used in multithreading reduces to a single value, this would also be inconsequential in languages with native parallelization.

It should be noted that the exact same values are produced whether flags are kept or not if the random state is not modulated.

Light GBM performs the same function as XGBoost but it's faster though more sensitive to over fitting. Let's see how it performs on a data set of this size (~1400 items)

lgb_noflag_test, lgb_noflag_diff = ttest_model(n = 50, modelConstructor = lgb.LGBMRegressor(objective='regression'), data0 = train_x_noflag_0, data1 = train_x_noflag_1, threads = nthreads)
print(lgb_noflag_diff)
print(lgb_noflag_test)

Again it seems we are stuck due to random seeds. It should be noted that this is much faster.

In [ ]:
from sklearn.model_selection import GridSearchCV

Text block below is for parameter search and has been converted to markdown to avoid being run

In [ ]:
xgb_params = {
         "learning_rate": np.linspace(0.032,0.033,3),
         "max_depth": [3],
         "n_estimators": [2000],
         "colsample_bytree": np.linspace(0.065,0.075,3),
         "gamma": [0.01],
         "silent": [1],
         "min_child_weight": [1],
         "reg_alpha": np.linspace(0.2,0.4,3),
         "reg_lambda": np.linspace(0.55,0.65,3),
         "subsample": [0.5],
         "nthread": [1]
         }

xgb0 = XGBRegressor()

###   Potential LONG RUN TIME    ### 
xgb_grid = GridSearchCV(xgb0,
                        xgb_params,
                        cv = 3,
                        scoring = "neg_mean_squared_error",
                        n_jobs = -1,
                        verbose=50)

xgb_grid.fit(train_x_noflag_1,train_y)

print(np.mean(rmse(xgb.XGBRegressor(**xgb_grid.best_params_),train_x_noflag_1,train_y)))
print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

lgbm_train = lgb.Dataset(train_x_noflag_1, label = train_y)

lgbm_params = {'objective' : ['regression'],
               'metric': ['mse'],
               'num_leaves' : [5,8,12],
               'learning_rate' : np.linspace(0.03,0.035,3),
               'lambda': np.linspace(0.01,0.03,3),
               'max_bin': [100],
               'bagging_fraction' : [0.01],
               'feature_fraction' : [0.173],
               'num_rounds' : [500],
               'n_estimators': [750],
               'sub_feature' : [0.5],
               'verbose': [0]
              }

lgbm_test = lgb.LGBMRegressor()

lgbm_grid =  GridSearchCV(lgbm_test,
                        lgbm_params,
                        scoring = "neg_mean_squared_error",
                        cv = 5,
                        n_jobs = -1,
                        verbose=50)

lgbm_grid.fit(train_x_noflag_1,train_y)

print(np.mean(rmse(lgb.LGBMRegressor(**lgbm_grid.best_params_),train_x_noflag_1,train_y)))
print(lgbm_grid.best_score_)
print(lgbm_grid.best_params_)

# Prediction and Submission

In [ ]:
trainLen = trainData.shape[0]

train_x, train_y = split_xy(featureFix(scalePrice(imputeVals_noflag_1(trainData))))
test_x = featureFix(imputeVals_noflag_1(testData))

withDummy = makeStr(pd.concat([train_x,test_x]))

train_x = withDummy.iloc[0:trainLen]
test_x = withDummy.iloc[trainLen:]

print(train_x.shape)

In [ ]:
xgb = XGBRegressor(colsample_bytree = 0.075, gamma = 0.01, learning_rate = 0.0325, max_depth = 3,
                  min_child_weight = 1, n_estimators = 2000, nthread = nthreads, reg_alpha = 0.2, reg_lambda = 0.65,
                  silent = 1, subsample = 0.5)

xgb.fit(train_x,train_y)

xgb_preds = xgb.predict(test_x)

In [ ]:
np.mean(rmse(xgb,train_x))

In [ ]:
lgbm_params = {'bagging_fraction': 0.1, 'feature_fraction': 0.1, 'lambda': 0.1, 'learning_rate': 0.0325, 'max_bin': 60, 'metric': 'mse', 'n_estimators': 750, 'num_leaves': 8, 'objective': 'regression', 'sub_feature': 0.5, 'verbose': 0}
lgbm = lgb.LGBMRegressor(**lgbm_params)
lgbm.fit(train_x,train_y)
lgbm_preds = lgbm.predict(test_x)

In [ ]:
lgbm_params = {'bagging_fraction': 0.1, 'feature_fraction': 0.1, 'lambda': 0.1, 'learning_rate': 0.0325, 'max_bin': 60, 'metric': 'mse', 'n_estimators': 750, 'num_leaves': 8, 'objective': 'regression', 'sub_feature': 0.5, 'verbose': 0}
lgbm2 = lgb.LGBMRegressor(**lgbm_params)
np.mean(rmse(lgbm2,train_x))


In [ ]:
submit_frame = pd.DataFrame()
submit_frame['Id'] = testID
submit_frame['SalePrice'] = invPrice(xgb_preds)
submit_frame.to_csv('submission.csv',index=False)

# Towards the future!

This performance with a single model is pretty good, but we can do better. To climb the leader boards, it will be necessary to implement an ensemble model.   
As a point of good practice, we should also have withheld a train-test split to test our models on in addition to using cross validation. To some extent this is done when you submit for ranking but it can't be done rapidly or in a manner that is conducive to good study of the model.

For now, this will suffice until I feel like looking at this data set again.


# Parameter notes

## XG Boost

In [ ]:
xgb_params1 = {
         "learning_rate": np.linspace(0.04,0.06,3),
         "max_depth": [3,5,8],
         "n_estimators": [2500],
         "colsample_bytree": np.linspace(0.4,0.6,3),
         "gamma": np.linspace(0.03,0.05,3),
         "silent": [1],
         "min_child_weight": [1],
         "reg_alpha": [0.5],
         "reg_lambda": [0.8],
         "subsample": [0.5],
         "nthread": [1]
         }
         
 # 0.9088979396366262
#{'colsample_bytree': 0.4, 'gamma': 0.05, 'learning_rate': 0.04, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 2500, 'nthread': 1, 'reg_alpha': 0.5, 'reg_lambda': 0.8, 'silent': 1, 'subsample': 0.5}

xgb_params2 = {
         "learning_rate": np.linspace(0.025,0.04,3),
         "max_depth": [3,4],
         "n_estimators": [2000,3000],
         "colsample_bytree": np.linspace(0.2,0.4,3),
         "gamma": np.linspace(0.05,0.075,3),
         "silent": [1],
         "min_child_weight": [1],
         "reg_alpha": [0.5],
         "reg_lambda": [0.8],
         "subsample": [0.5],
         "nthread": [1]
         }
         
# 0.9117986931609089
# {'colsample_bytree': 0.2, 'gamma': 0.05, 'learning_rate': 0.0325, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 2000, 'nthread': 1, 'reg_alpha': 0.5, 'reg_lambda': 0.8, 'silent': 1, 'subsample': 0.5}

xgb_params3 = {
         "learning_rate": np.linspace(0.0275,0.035,4),
         "max_depth": [3],
         "n_estimators": [2000,2500],
         "colsample_bytree": np.linspace(0.05,0.25,4),
         "gamma": np.linspace(0.01,0.075,3),
         "silent": [1],
         "min_child_weight": [1],
         "reg_alpha": [0.5],
         "reg_lambda": [0.8],
         "subsample": [0.5],
         "nthread": [1]
         }

# 0.9149185558702502
# {'colsample_bytree': 0.05, 'gamma': 0.01, 'learning_rate': 0.0325, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 2000, 'nthread': 1, 'reg_alpha': 0.5, 'reg_lambda': 0.8, 'silent': 1, 'subsample': 0.5}

xgb_params4 = {
         "learning_rate": [0.0325],
         "max_depth": [3],
         "n_estimators": [1750,2000],
         "colsample_bytree": np.linspace(0.01,0.1,4),
         "gamma": [0.01],
         "silent": [1],
         "min_child_weight": [1],
         "reg_alpha": np.linspace(0.25,0.75,5),
         "reg_lambda": np.linspace(0.7,0.9,3),
         "subsample": [0.5],
         "nthread": [1]
         }

# 0.9137795024094789
# {'colsample_bytree': 0.07, 'gamma': 0.01, 'learning_rate': 0.0325, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 2000, 'nthread': 1, 'reg_alpha': 0.375, 'reg_lambda': 0.7, 'silent': 1, 'subsample': 0.5}

xgb_params4 = {
         "learning_rate": [0.0325],
         "max_depth": [3],
         "n_estimators": [1900,2100],
         "colsample_bytree": np.linspace(0.65,0.75,3),              # There was a mistake on the order of magnitude for these values in this run
         "gamma": [0.01],
         "silent": [1],
         "min_child_weight": [1],
         "reg_alpha": np.linspace(0.3,0.5,5),
         "reg_lambda": np.linspace(0.6,0.75,4),
         "subsample": np.linspace(0.4,0.5,3),
         "nthread": [1]
         }

# 0.9106672141801405
# {'colsample_bytree': 0.65, 'gamma': 0.01, 'learning_rate': 0.0325, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 1900, 'nthread': 1, 'reg_alpha': 0.3, 'reg_lambda': 0.6, 'silent': 1, 'subsample': 0.5}

xgb_params5 = {
         "learning_rate": np.linspace(0.032,0.033,3),
         "max_depth": [3],
         "n_estimators": [2000],
         "colsample_bytree": np.linspace(0.065,0.075,3),
         "gamma": [0.01],
         "silent": [1],
         "min_child_weight": [1],
         "reg_alpha": np.linspace(0.2,0.4,3),
         "reg_lambda": np.linspace(0.55,0.65,3),
         "subsample": [0.5],
         "nthread": [1]
         }
# 0.9118781247040025
# {'colsample_bytree': 0.075, 'gamma': 0.01, 'learning_rate': 0.0325, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 2000, 'nthread': 1, 'reg_alpha': 0.2, 'reg_lambda': 0.65, 'silent': 1, 'subsample': 0.5}


## Light GBM

In [ ]:
lgbm_params1 = {'objective' : ['regression'],
               'metric': ['mse'],
               'num_leaves' : [3,5,8],
               'learning_rate' : np.linspace(0.01,0.1,5),
               'lambda': np.linspace(0.1,0.9,5),
               'max_bin': [40,50,60],
               'bagging_fraction' : np.linspace(0.1,0.9,2),
               'feature_fraction' : np.linspace(0.1,0.9,2),
               'n_estimators': [750,1000],
               'sub_feature' : [0.5],
               'verbose': [0]
              }

# 0.9012716484076563
# {'bagging_fraction': 0.1, 'feature_fraction': 0.1, 'lambda': 0.1, 'learning_rate': 0.0325, 'max_bin': 60, 'metric': 'mse', 'n_estimators': 750, 'num_leaves': 8, 'objective': 'regression', 'sub_feature': 0.5, 'verbose': 0}

lgbm_params2 = {'objective' : ['regression'],
               'metric': ['mse'],
               'num_leaves' : [10,50,100],
               'learning_rate' : [0.0325],
               'lambda': np.linspace(0.1,0.5,3),
               'max_bin': [60,100],
               'bagging_fraction' : np.linspace(0.05,0.15,3),
               'feature_fraction' : np.linspace(0.05,0.15,3),
               'n_estimators': [750],
               'sub_feature' : [0.5],
               'verbose': [0]
              }

# 0.9007668429828806
# {'bagging_fraction': 0.05, 'feature_fraction': 0.1, 'lambda': 0.1, 'learning_rate': 0.0325, 'max_bin': 60, 'metric': 'mse', 'n_estimators': 750, 'num_leaves': 10, 'objective': 'regression', 'sub_feature': 0.5, 'verbose': 0}

lgbm_params3 = {'objective' : ['regression'],
               'metric': ['mse'],
               'num_leaves' : [8,10,15],
               'learning_rate' : [0.0325],
               'lambda': np.linspace(0.01,0.1,5),
               'max_bin': [60,100],
               'bagging_fraction' : [0.05],
               'feature_fraction' : [0.1],
               'num_rounds' : [50,100,500],
               'n_estimators': [750],
               'sub_feature' : [0.5],
               'verbose': [0]
              }

# 0.9007548589509198
# {'bagging_fraction': 0.05, 'feature_fraction': 0.1, 'lambda': 0.01, 'learning_rate': 0.0325, 'max_bin': 60, 'metric': 'mse', 'n_estimators': 750, 'num_leaves': 8, 'num_rounds': 500, 'objective': 'regression', 'sub_feature': 0.5, 'verbose': 0}

lgbm_params4 = {'objective' : ['regression'],
               'metric': ['mse'],
               'num_leaves' : [8],
               'learning_rate' : np.linspace(0.030,0.035,5),
               'lambda': np.linspace(0.005,0.01,5),
               'max_bin': [60,100],
               'bagging_fraction' : [0.05],
               'feature_fraction' : [0.1],
               'num_rounds' : [500],
               'n_estimators': [750],
               'sub_feature' : [0.5],
               'verbose': [0]
              }

# 0.9008893309397983
# {'bagging_fraction': 0.05, 'feature_fraction': 0.1, 'lambda': 0.005, 'learning_rate': 0.03375, 'max_bin': 60, 'metric': 'mse', 'n_estimators': 750, 'num_leaves': 8, 'num_rounds': 500, 'objective': 'regression', 'sub_feature': 0.5, 'verbose': 0}

lgbm_params5 = {'objective' : ['regression'],
               'metric': ['mse'],
               'num_leaves' : [8],
               'learning_rate' : np.linspace(0.033,0.035,5),
               'lambda': np.linspace(0.00,0.01,5),
               'max_bin': [60,100],
               'bagging_fraction' : [0.05],
               'feature_fraction' : [0.1],
               'num_rounds' : [500],
               'n_estimators': [750],
               'sub_feature' : [0.5],
               'verbose': [0]
              }
# 0.9009788735426308
# {'bagging_fraction': 0.05, 'feature_fraction': 0.1, 'lambda': 0.0, 'learning_rate': 0.034, 'max_bin': 60, 'metric': 'mse', 'n_estimators': 750, 'num_leaves': 8, 'num_rounds': 500, 'objective': 'regression', 'sub_feature': 0.5, 'verbose': 0}


lgbm_params6 = {'objective' : ['regression'],
               'metric': ['mse'],
               'num_leaves' : [8,100],
               'learning_rate' : [0.001,0.01,0.03,0.1],
               'lambda': np.linspace(0.02,0.9,5),
               'max_bin': [20,60,100],
               'bagging_fraction' : np.linspace(0.01,0.9,4),
               'feature_fraction' : np.linspace(0.01,0.5,4),
               'num_rounds' : [500],
               'n_estimators': [750],
               'sub_feature' : [0.5],
               'verbose': [0]
              }

#  0.29463939175444454
# -0.09921780759203461
# {'bagging_fraction': 0.01, 'feature_fraction': 0.17333333333333334, 'lambda': 0.02, 'learning_rate': 0.03, 'max_bin': 100, 'metric': 'mse', 'n_estimators': 750, 'num_leaves': 8, 'num_rounds': 500, 'objective': 'regression', 'sub_feature': 0.5, 'verbose': 0}

lgbm_params7 = {'objective' : ['regression'],
               'metric': ['mse'],
               'num_leaves' : [5,8,12],
               'learning_rate' : np.linspace(0.03,0.035,3),
               'lambda': np.linspace(0.01,0.03,3),
               'max_bin': [100],
               'bagging_fraction' : [0.01],
               'feature_fraction' : [0.173],
               'num_rounds' : [500],
               'n_estimators': [750],
               'sub_feature' : [0.5],
               'verbose': [0]
              }

#  0.2952754856025843
# -0.09286440091369501
# {'bagging_fraction': 0.01, 'feature_fraction': 0.173, 'lambda': 0.01, 'learning_rate': 0.0325, 'max_bin': 100, 'metric': 'mse', 'n_estimators': 750, 'num_leaves': 12, 'num_rounds': 500, 'objective': 'regression', 'sub_feature': 0.5, 'verbose': 0}